<a href="https://colab.research.google.com/github/audalsgh/20250812-14/blob/main/0812_Roboflow_Segformer2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ========================
# 0) 필수 라이브러리 설치 (transformers 4.x 고정)
# ========================
!pip install -q "transformers>=4.44,<5" accelerate evaluate opencv-python-headless pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.2 MB/s eta 0:00:00


In [2]:
# ========================
# 1) Roboflow ZIP 업로드
# ========================
from google.colab import files
up = files.upload()  # Roboflow에서 받은 dataset.zip 선택
ZIP_PATH = "/content/" + list(up.keys())[0]

# ========================
# 2) 압축 해제
# ========================
import os, zipfile, shutil

EXTRACT_DIR = "/content/ds_rf"
if os.path.isdir(EXTRACT_DIR):
    shutil.rmtree(EXTRACT_DIR)
os.makedirs(EXTRACT_DIR, exist_ok=True)

with zipfile.ZipFile(ZIP_PATH, "r") as z:
    z.extractall(EXTRACT_DIR)

print("unzipped to", EXTRACT_DIR, "->", os.listdir(EXTRACT_DIR))

# ========================
# 3) 데이터 구조 파악
# ========================
import os

def find_split_dir(root, names=("train","valid","val","test")):
    found = {}
    for n in names:
        p = os.path.join(root, n)
        if os.path.isdir(p):
            found["valid" if n in ("valid","val") else n] = p
    return found

splits = find_split_dir(EXTRACT_DIR)
if not splits:
    raise RuntimeError("train/valid/test 폴더를 찾지 못함. ZIP 내용 확인")

print("splits:", splits)

# ========================
# 4) 학습 클래스 설정 (멀티클래스 5종)
# ========================
COLLAPSE_TO_BINARY = False
CLASS_NAMES = ["background","lane","lane_dot","lane_mid","lane_crosswalk"]
NUM_LABELS = len(CLASS_NAMES)
id2label = {i:n for i,n in enumerate(CLASS_NAMES)}
label2id = {n:i for i,n in id2label.items()}
print("NUM_LABELS:", NUM_LABELS, id2label)

Saving ---.v1i.png-mask-semantic.zip to ---.v1i.png-mask-semantic.zip
unzipped to /content/ds_rf -> ['README.roboflow.txt', 'README.dataset.txt', 'valid', 'test', 'train']
splits: {'train': '/content/ds_rf/train', 'valid': '/content/ds_rf/valid', 'test': '/content/ds_rf/test'}
NUM_LABELS: 5 {0: 'background', 1: 'lane', 2: 'lane_dot', 3: 'lane_mid', 4: 'lane_crosswalk'}


In [3]:
# ========================
# 5) RFSegFolder (견고/멀티클래스, 라벨 안전 정규화)
# ========================
# === 멀티클래스용 RFSegFolder (라벨 안전 정규화)만 남기세요 ===
import os, glob, re
import numpy as np
from PIL import Image
from torch.utils.data import Dataset

_SUFFIX_RE = re.compile(r'(_|-)(mask|masks|label|labels|seg|segment|segmentation)$', re.I)
def _stem_no_suffix(path):
    s = os.path.splitext(os.path.basename(path))[0]
    s = _SUFFIX_RE.sub('', s)
    return s

def _is_img(name):
    return name.lower().endswith((".jpg",".jpeg",".png",".bmp",".tif",".tiff"))

class RFSegFolder(Dataset):
    def __init__(self, split_dir, processor, num_labels=5, ignore_index=255):
        self.split_dir = split_dir
        self.processor = processor
        self.num_labels = num_labels
        self.ignore_index = ignore_index

        img_cands = [os.path.join(split_dir, "images"), split_dir]
        self.img_dir = None
        for d in img_cands:
            if os.path.isdir(d) and any(_is_img(f) for f in os.listdir(d)):
                self.img_dir = d; break
        if self.img_dir is None:
            raise RuntimeError(f"No images found in {split_dir}")

        mask_cands = ["masks","labels","annotations","masks_png","labels_png","mask",".","Labels","Masks"]
        self.mask_dirs = []
        for c in mask_cands:
            d = split_dir if c == "." else os.path.join(split_dir, c)
            if os.path.isdir(d): self.mask_dirs.append(d)
        if not self.mask_dirs:
            for root, dirs, files in os.walk(split_dir):
                if os.path.abspath(root) == os.path.abspath(self.img_dir): continue
                if any(f.lower().endswith(".png") for f in files):
                    self.mask_dirs.append(root)
            if not self.mask_dirs: self.mask_dirs = [split_dir]

        mask_map = {}
        for md in self.mask_dirs:
            for p in glob.glob(os.path.join(md, "*.png")):
                mask_map[_stem_no_suffix(p)] = p

        self.items = []
        for ip in sorted(glob.glob(os.path.join(self.img_dir, "*.*"))):
            if not _is_img(ip): continue
            st = _stem_no_suffix(ip)
            mp = mask_map.get(st)
            if mp and os.path.exists(mp):
                self.items.append((ip, mp))
        if not self.items:
            raise RuntimeError(f"No (image,mask) pairs in {split_dir}")
        print(f"✅ {split_dir}: {len(self.items)} pairs")

    def _load_mask_indexed(self, mp):
        img = Image.open(mp)
        if img.mode in ("P","L"):
            m = np.array(img, dtype=np.int64)
        else:
            m = np.array(img.convert("L"), dtype=np.int64)

        # ★ 핵심: 범위 밖은 전부 255(무시)로 보냄 → device assert 방지
        bad = m > (self.num_labels - 1)
        m[bad] = self.ignore_index
        m[m < 0] = self.ignore_index
        return m

    def __len__(self): return len(self.items)

    def __getitem__(self, idx):
        ip, mp = self.items[idx]
        image = Image.open(ip).convert("RGB")
        mask  = self._load_mask_indexed(mp)  # (H,W) int64 with {0..K-1, 255}
        enc = self.processor(images=image, segmentation_maps=mask, return_tensors="pt")
        return {k: v.squeeze(0) for k, v in enc.items()}

# ★ 꼭 재생성: 이전에 만든 train_ds/val_ds를 덮어씁니다.
train_ds = RFSegFolder(train_dir, processor, num_labels=NUM_LABELS, ignore_index=255)
val_ds   = RFSegFolder(valid_dir, processor, num_labels=NUM_LABELS, ignore_index=255)
print(f"train={len(train_ds)}, valid={len(val_ds)}")

# ========================
# 6) 프로세서/모델 로드
# ========================
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation
import torch

CKPT = "nvidia/segformer-b0-finetuned-ade-512-512"

processor = SegformerImageProcessor.from_pretrained(
    CKPT,
    reduce_labels=False  # 라벨 줄임 비활성화(우리 인덱스 유지)
)

model = SegformerForSemanticSegmentation.from_pretrained(
    CKPT,
    num_labels=NUM_LABELS,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

print("Model num_labels:", model.config.num_labels)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/image_processing_base.py:412: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type', 'reduce_labels'
  image_processor = cls(**image_processor_dict)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/15.0M [00:00<?, ?B/s]

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([5]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([5, 256, 1, 1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model num_labels: 5


In [4]:
# ========================
# 7) 데이터셋 생성
# ========================
train_dir = splits.get("train")
valid_dir = splits.get("valid") or splits.get("val") or train_dir

if train_dir is None:
    raise RuntimeError("train 폴더를 찾지 못했습니다. ZIP 구조를 확인하세요.")

train_ds = RFSegFolder(train_dir, processor, num_labels=NUM_LABELS, ignore_index=255)
val_ds   = RFSegFolder(valid_dir, processor, num_labels=NUM_LABELS, ignore_index=255)
print(f"✅ Dataset ready: train={len(train_ds)}, valid={len(val_ds)}")

✅ /content/ds_rf/train: paired 1422 samples
✅ /content/ds_rf/valid: paired 70 samples
✅ Dataset ready: train=1422, valid=70


In [5]:
# ========================
# 8) 학습 + 저장 + 다운로드
# ========================
from transformers import TrainingArguments, Trainer
import numpy as np, evaluate, torch, os, zipfile
from google.colab import files

metric = evaluate.load("mean_iou")

def _to_py(o):
    if isinstance(o, np.ndarray):
        return o.tolist()
    if isinstance(o, (np.floating, np.integer)):
        return o.item()
    return o

def compute_metrics(eval_pred):
    logits, labels = eval_pred  # logits: (N, C, h', w'), labels: (N, H, W)
    if isinstance(logits, tuple):
        logits = logits[0]
    lt = torch.from_numpy(logits)
    yt = torch.from_numpy(labels)

    lt_up = torch.nn.functional.interpolate(
        lt, size=yt.shape[-2:], mode="bilinear", align_corners=False
    )
    preds = lt_up.argmax(dim=1).cpu().numpy()

    res = metric.compute(
        predictions=preds,
        references=labels,
        num_labels=NUM_LABELS,
        ignore_index=255,
        reduce_labels=False,
    )
    return {k: _to_py(v) for k, v in res.items()}

args = TrainingArguments(
    output_dir="segformer-lane",
    learning_rate=5e-5,
    num_train_epochs=20,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="epoch",   # ★ transformers 4.x는 evaluation_strategy
    save_strategy="epoch",
    fp16=False,                     # 안정 우선(원하면 True로 변경 가능)
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="mean_iou",
    greater_is_better=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
)

trainer.train()

# 베스트 저장
BEST_DIR = "segformer-lane/best"
os.makedirs(BEST_DIR, exist_ok=True)
trainer.save_model(BEST_DIR)
processor.save_pretrained(BEST_DIR)

print("✅ Saved best to:", BEST_DIR)

# 아티팩트 압축(zip) 후 다운로드
ZIP_OUT = "segformer_lane_best.zip"
with zipfile.ZipFile(ZIP_OUT, "w", zipfile.ZIP_DEFLATED) as z:
    for fname in ["config.json", "preprocessor_config.json", "model.safetensors", "pytorch_model.bin"]:
        p = os.path.join(BEST_DIR, fname)
        if os.path.exists(p):
            z.write(p, arcname=os.path.join("best", fname))
    for extra in ["trainer_state.json", "trainer_config.json", "all_results.json"]:
        p = os.path.join("segformer-lane", extra)
        if os.path.exists(p):
            z.write(p, arcname=os.path.join("run_meta", extra))

print("📦 Zip created:", ZIP_OUT)
files.download(ZIP_OUT)

Epoch,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Per Category Iou,Per Category Accuracy
1,0.220700,0.189557,0.302947,0.318986,0.951626,"[0.9533404299716234, 0.2097740412321306, nan, 0.04586653164782338, 0.0028063144153095887]","[0.9999103982537487, 0.22629283332915548, nan, 0.04693513858206746, 0.0028075633677450483]"
2,0.097000,0.099698,0.593844,0.646145,0.970974,"[0.973023448731079, 0.5661042480679055, nan, 0.41693554224966134, 0.4193137409278956]","[0.9956649519012045, 0.6424736169252752, nan, 0.45910007060056024, 0.4873430884028916]"
3,0.083300,0.074150,0.704333,0.769853,0.977377,"[0.9773057241102682, 0.6295639196384744, nan, 0.6300455362429718, 0.5804171633737047]","[0.9933997041408147, 0.716940315343544, nan, 0.6845749163041746, 0.684496427115655]"
4,0.063900,0.063101,0.728608,0.821513,0.978451,"[0.9784024379868222, 0.6580576252975439, nan, 0.6731531790689081, 0.6048185294851647]","[0.9905083425006518, 0.7768405484671497, nan, 0.772754446696577, 0.7459467103676036]"
5,0.057300,0.057997,0.736318,0.810105,0.979429,"[0.9789706359587732, 0.6414031988419979, nan, 0.705365013272525, 0.6195350397622822]","[0.9927836487796531, 0.6953275010653498, nan, 0.7995798127946435, 0.7527306153347032]"
6,0.049600,0.054754,0.749627,0.840209,0.980158,"[0.9798640585013739, 0.6770082088334282, nan, 0.7105287336312114, 0.6311050627517286]","[0.9909390089584404, 0.76947083448224, nan, 0.8365853241932177, 0.7638402475646987]"
7,0.047000,0.052908,0.750007,0.834697,0.980446,"[0.9802266194947085, 0.6718687112168671, nan, 0.7194866985275183, 0.6284462223838531]","[0.9914338996356389, 0.7615747123555511, nan, 0.8670147350201554, 0.7187653376146942]"
8,0.045000,0.050919,0.756240,0.838984,0.980671,"[0.9800068153650205, 0.680066904029488, nan, 0.7355933225914297, 0.6292931042093218]","[0.9916654767940148, 0.7603288797533402, nan, 0.8330666833678745, 0.7708737137200422]"
9,0.039900,0.050582,0.756528,0.850149,0.980546,"[0.9802323218035324, 0.6761148856805049, nan, 0.7380090732147747, 0.6317563632384657]","[0.9902809274879081, 0.8142054997117288, nan, 0.8516648067594342, 0.7444451838850022]"
10,0.041300,0.049598,0.760376,0.853565,0.980745,"[0.9803618443023611, 0.6785111258210703, nan, 0.7450378012086273, 0.637593857379205]","[0.9901911523189221, 0.8350637956533727, nan, 0.8388229064656448, 0.7501809318614769]"


/usr/local/lib/python3.11/dist-packages/datasets/features/image.py:357: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
/usr/local/lib/python3.11/dist-packages/datasets/features/image.py:357: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_

✅ Saved best to: segformer-lane/best
📦 Zip created: segformer_lane_best.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# ▶ 영상 업로드
from google.colab import files
up = files.upload()  # 로컬에서 mp4 등 선택
VIDEO_IN = "/content/" + list(up.keys())[0]
print("입력 영상:", VIDEO_IN)

Saving KakaoTalk_20250707_100128756.mp4 to KakaoTalk_20250707_100128756.mp4
입력 영상: /content/KakaoTalk_20250707_100128756.mp4


In [7]:
# ▶ SegFormer 추론 + 컬러 오버레이 (5클래스)
import os, cv2, numpy as np, torch
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation
from google.colab import files

MODEL_DIR = "/content/segformer-lane/best"
VIDEO_OUT = "/content/out_lane_overlay.mp4"
ALPHA = 0.5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

processor = SegformerImageProcessor.from_pretrained(MODEL_DIR)
model = SegformerForSemanticSegmentation.from_pretrained(MODEL_DIR).to(DEVICE).eval()
print("num_labels:", model.config.num_labels, "id2label:", getattr(model.config, "id2label", None))

PALETTE = [
    (0, 0, 0),        # background
    (0, 255, 0),      # lane
    (0, 165, 255),    # lane_dot
    (255, 0, 0),      # lane_mid
    (255, 255, 255),  # lane_crosswalk
]
PALETTE = PALETTE[: model.config.num_labels]  # 모델 클래스 수에 맞춤

cap = cv2.VideoCapture(VIDEO_IN)
assert cap.isOpened(), f"Cannot open video: {VIDEO_IN}"
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
w   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(VIDEO_OUT, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

with torch.no_grad():
    idx = 0
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        inputs = processor(images=rgb, return_tensors="pt").to(DEVICE)
        logits = model(**inputs).logits
        logits = torch.nn.functional.interpolate(logits, size=(h, w), mode="bilinear", align_corners=False)
        pred = logits.argmax(dim=1)[0].detach().cpu().numpy().astype(np.int32)

        overlay = np.zeros_like(frame)
        for cls_id in range(1, model.config.num_labels):  # 0=배경 제외
            mask = (pred == cls_id)
            if mask.any():
                overlay[mask] = PALETTE[cls_id]

        blended = cv2.addWeighted(frame, 1.0-ALPHA, overlay, ALPHA, 0.0)
        out.write(blended)
        idx += 1
        if idx % 50 == 0:
            print(f"Processed {idx} frames...")

cap.release(); out.release()
print("✅ Saved video:", VIDEO_OUT)
files.download(VIDEO_OUT)

num_labels: 5 id2label: {0: 'background', 1: 'lane', 2: 'lane_dot', 3: 'lane_mid', 4: 'lane_crosswalk'}
Processed 50 frames...
Processed 100 frames...
Processed 150 frames...
Processed 200 frames...
Processed 250 frames...
Processed 300 frames...
Processed 350 frames...
Processed 400 frames...
Processed 450 frames...
Processed 500 frames...
Processed 550 frames...
Processed 600 frames...
Processed 650 frames...
Processed 700 frames...
Processed 750 frames...
Processed 800 frames...
Processed 850 frames...
Processed 900 frames...
✅ Saved video: /content/out_lane_overlay.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

깃허브 사본저장 오류 해결 코드

In [8]:
# === 0) 준비 ===
REPO_URL = "https://github.com/audalsgh/20250812.git"
ROOT = "/content/20250812"

!rm -rf "$ROOT"
!git clone --depth 1 "$REPO_URL" "$ROOT"

!pip -q install nbformat nbconvert

import os, glob, json
import nbformat as nbf
from nbformat.validator import validate, ValidationError

def strip_widgets_everywhere(nb):
    """루트/셀/출력 전역에서 위젯 관련 메타데이터와 출력 제거"""
    # 1) 루트 metadata 정리
    if "widgets" in nb.metadata:
        # 완전히 제거하는 편이 안전
        nb.metadata.pop("widgets", None)

    # 권장: kernelspec / language_info 기본값이 없으면 채워줌
    nb.metadata.setdefault("kernelspec", {"name":"python3","display_name":"Python 3"})
    nb.metadata.setdefault("language_info", {"name":"python"})

    # 2) 각 셀 순회
    for cell in nb.cells:
        md = cell.get("metadata", {})
        md.pop("widgets", None)   # 셀의 widget 메타데이터 제거
        cell["metadata"] = md

        # 3) 출력에서 위젯 display 제거
        if cell.get("cell_type") == "code":
            outs = []
            for out in cell.get("outputs", []):
                data = out.get("data", {})
                if isinstance(data, dict) and "application/vnd.jupyter.widget-view+json" in data:
                    # 위젯 출력은 폐기
                    continue
                outs.append(out)
            cell["outputs"] = outs

    # 4) nbformat 필수 필드 보정
    nb.nbformat = 4
    nb.nbformat_minor = max(getattr(nb, "nbformat_minor", 5), 5)

    return nb

# === 1) 대상 노트북 찾기 ===
targets = glob.glob(os.path.join(ROOT, "**", "*.ipynb"), recursive=True)
print(f"Found {len(targets)} notebooks:")
for p in targets:
    print(" -", p)

if not targets:
    raise SystemExit("❌ 저장소에 ipynb가 없습니다. 경로/브랜치를 확인하세요.")

# === 2) 소독 + 검증 + 덮어쓰기 ===
cleaned = []
failed = []
for path in targets:
    try:
        nb = nbf.read(path, as_version=4)
        nb = strip_widgets_everywhere(nb)

        # 유효성 검사
        try:
            validate(nb)
        except ValidationError as e:
            print(f"[warn] validate error in {os.path.relpath(path, ROOT)}: {e}")

        # 원래 이름 그대로 덮어쓰기(★ GitHub에서 같은 파일명으로 교체되어야 미리보기 갱신)
        nbf.write(nb, path)
        cleaned.append(path)
    except Exception as e:
        failed.append((path, str(e)))

print(f"\n✅ cleaned: {len(cleaned)}  ❌ failed: {len(failed)}")
for p, msg in failed:
    print("  -", os.path.relpath(p, ROOT), "->", msg)

# === 3) 출력/기타 메타데이터까지 싹 비운 '클린' 사본도 생성(선택) ===
# 원본 덮어쓰기는 이미 했고, 참고용으로 _clean 사본 추가
for path in targets:
    out_clean = os.path.splitext(path)[0] + "_clean.ipynb"
    !jupyter nbconvert --to notebook \
      --ClearOutputPreprocessor.enabled=True \
      --ClearMetadataPreprocessor.enabled=True \
      --output "{out_clean}" \
      "{path}"

print("\n📦 압축해서 내려받기…")
import zipfile
from google.colab import files

ZIP_OUT = "/content/sanitized_notebooks.zip"
with zipfile.ZipFile(ZIP_OUT, "w", zipfile.ZIP_DEFLATED) as z:
    for p in cleaned:
        z.write(p, arcname=os.path.relpath(p, ROOT))
    # 원하면 _clean 사본도 같이 묶기
    for p in glob.glob(os.path.join(ROOT, "**", "*_clean.ipynb"), recursive=True):
        z.write(p, arcname=os.path.relpath(p, ROOT))

print("✅ Created:", ZIP_OUT)
files.download(ZIP_OUT)

Cloning into '/content/20250812'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (5/5), 32.67 KiB | 16.33 MiB/s, done.
Found 2 notebooks:
 - /content/20250812/0813_runpod_Roboflow_Segformer.ipynb
 - /content/20250812/0812_Colab_Roboflow_Segformer_fixed.ipynb

✅ cleaned: 2  ❌ failed: 0
[NbConvertApp] Converting notebook /content/20250812/0813_runpod_Roboflow_Segformer.ipynb to notebook
[NbConvertApp] Writing 8713 bytes to /content/20250812/0813_runpod_Roboflow_Segformer_clean.ipynb
[NbConvertApp] Converting notebook /content/20250812/0812_Colab_Roboflow_Segformer_fixed.ipynb to notebook
[NbConvertApp] Writing 15782 bytes to /content/20250812/0812_Colab_Roboflow_Segformer_fixed_clean.ipynb

📦 압축해서 내려받기…
✅ Created: /content/sanitized_notebooks.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>